# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [4]:
#!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')  # Adjust the path as needed


In [7]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2)


In [11]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [12]:
chain = prompt | llm

In [13]:
product = "Rubber ducks"
chain.invoke(product)

AIMessage(content="Introducing the ultimate bath time companion - the Rubber Duck! This quacky little friend is perfect for adding some fun to your daily routine. With its waterproof design and squeaky sound, it's sure to make a splash in your tub. Say goodbye to boring baths and hello to endless entertainment with the Rubber Duck!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 27, 'total_tokens': 90, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aa73c76a-cab0-47c9-9654-c8510c697734-0', usage_metadata={'input_tokens': 27, 'output_tokens': 63, 'total_tokens': 90, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## SimpleSequentialChain

In [14]:
from langchain.chains import SimpleSequentialChain

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Initialize the language model with a higher temperature for creativity
llm = ChatOpenAI(temperature=0.9)

# Define the prompt template
first_prompt = ChatPromptTemplate.from_template(
    "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Create the chain
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# Run the chain with a specific product as input
response = chain_one.run({"product": "Superfast Blender 3000"})

# Print the response
print(response)


C:\Users\mktmi\AppData\Local\Temp\ipykernel_38408\3540288751.py:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain_one.run({"product": "Superfast Blender 3000"})


Introducing the Superfast Blender 3000 - the Usain Bolt of kitchen appliances! This bad boy can blend up a storm faster than you can say "smoothie". With its turbocharged motor and sleek design, you'll be whipping up delicious drinks and sauces in no time. Say goodbye to lumps and chunks, and hello to silky smooth perfection. Get ready to blend like a boss with the Superfast Blender 3000!


In [18]:
# Define the second prompt template
second_prompt = ChatPromptTemplate.from_template(
    "Now that you have seen the product description: {description}, "
    "please write a short review of this product."
)

# Chain 2, using the second prompt
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Run the second chain, passing the output of the first chain as input
product_description = "A funny and creative description of a blender."  # Example output from chain_one
response = chain_two.run({"description": product_description})

# Print the response
print(response)


I recently purchased this hilarious blender and it has been a game-changer in my kitchen! Not only does it blend like a dream, but it also brings a smile to my face every time I use it. The witty description on the packaging never fails to make me laugh. Plus, the functionality and durability of the blender are top-notch. I highly recommend this product to anyone in need of a reliable and entertaining kitchen appliance. Trust me, you won't be disappointed!


In [19]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [20]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing our quacking good rubber ducks - they float, they squeak, and they'll add some serious fun to your bath time! These little ducky darlings are perfect for kids and adults alike, bringing a touch of whimsy to your daily routine. So why settle for a boring old bath when you can make a splash with our adorable rubber ducks? Get yours today and let the good times float! 🦆🛁 #RubberDuckyYoureTheOne
I absolutely love my quacking good rubber ducks! They are so cute and fun to play with in the bath. The fact that they float and squeak adds an extra element of enjoyment to my bath time routine. Not only are they great for kids, but they bring a smile to my face every time I use them. I highly recommend these adorable rubber ducks to anyone looking to add some fun to their bath time. #RubberDuckyYoureTheOne 🦆🛁

> Finished chain.


'I absolutely love my quacking good rubber ducks! They are so cute and fun to play with in the bath. The fact that they float and squeak adds an extra element of enjoyment to my bath time routine. Not only are they great for kids, but they bring a smile to my face every time I use them. I highly recommend these adorable rubber ducks to anyone looking to add some fun to their bath time. #RubberDuckyYoureTheOne 🦆🛁'

**Repeat the above twice for different products**

In [21]:
# Different Product 1
from langchain.chains import SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Initialize the language model with a higher temperature for creativity
llm = ChatOpenAI(temperature=0.9)

# Define the first prompt template for the product description
first_prompt = ChatPromptTemplate.from_template(
    "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise."
)

# Define the second prompt template for the product review
second_prompt = ChatPromptTemplate.from_template(
    "Now that you have seen the product description: {description}, "
    "please write a short review of this product."
)

# Create the chains
chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Function to run the chains for a given product
def run_for_product(product):
    # Run the first chain to get the product description
    product_description = chain_one.run({"product": product})
    print(f"Product Description for {product}: {product_description}")
    
    # Run the second chain to get the product review based on the description
    review = chain_two.run({"description": product_description})
    print(f"Review for {product}: {review}")

# List of products
products = ["Superfast Blender 3000", "Magic Coffee Maker X"]

# Run the chains for each product
for product in products:
    run_for_product(product)


Product Description for Superfast Blender 3000: Introducing the Superfast Blender 3000 - the turbocharged kitchen gadget that will have you blending faster than the speed of light (almost)!

With its powerful motor and stainless steel blades, this blender will pulverize your ingredients in seconds, turning even the toughest veggies into silky smooth soups and delicious smoothies.

Say goodbye to lumps and chunks with the Superfast Blender 3000 - it's like having your own personal smoothie superhero in the kitchen!
Review for Superfast Blender 3000: I absolutely love the Superfast Blender 3000! It truly lives up to its name as it blends ingredients in record time. The powerful motor and stainless steel blades make quick work of even the toughest veggies, leaving me with perfectly smooth soups and smoothies every time. Say goodbye to lumps and chunks with this blender - it's like having a superhero in the kitchen! I highly recommend the Superfast Blender 3000 to anyone looking for a fast

In [22]:
from langchain.chains import SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Initialize the language model with a higher temperature for creativity
llm = ChatOpenAI(temperature=0.9)

# Define the first prompt template for the product description
first_prompt = ChatPromptTemplate.from_template(
    "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise."
)

# Define the second prompt template for the product review
second_prompt = ChatPromptTemplate.from_template(
    "Now that you have seen the product description: {description}, "
    "please write a short review of this product."
)

# Create the chains
chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Function to run the chains for a given product
def run_for_product(product):
    # Run the first chain to get the product description
    product_description = chain_one.run({"product": product})
    print(f"Product Description for {product}: {product_description}")
    
    # Run the second chain to get the product review based on the description
    review = chain_two.run({"description": product_description})
    print(f"Review for {product}: {review}")

# List of products (including the Piaggio Vespa)
products = ["Piaggio Vespa"]

# Run the chains for each product
for product in products:
    run_for_product(product)


Product Description for Piaggio Vespa: Introducing the Piaggio Vespa - the scooter that's so stylish, even your grandma will want one! With its sleek design and smooth ride, you'll be the envy of all your friends as you zip around town in style. Plus, with its fuel-efficient engine and easy maneuverability, you'll never have to worry about being stuck in traffic again. So why settle for a boring old bike when you can ride in Italian-inspired luxury with the Piaggio Vespa? Get yours today and start turning heads wherever you go!
Review for Piaggio Vespa: I recently purchased the Piaggio Vespa and I couldn't be happier with my decision. Not only is it incredibly stylish and eye-catching, but it also provides a smooth and efficient ride. The fuel-efficient engine is a great bonus, as I can zip around town without constantly worrying about filling up. The maneuverability of the scooter is top-notch, making it easy to navigate through traffic with ease. Overall, I highly recommend the Piagg

## SequentialChain

In [49]:
from langchain.chains import SequentialChain

In [50]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='fr_review' #Give a name to your output
                    )

In [51]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='fr_short_review' #give a name to this output
                    )

In [52]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )

In [53]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )

In [54]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [55]:
review = df.Review[5]
overall_chain(review)

C:\Users\mktmi\AppData\Local\Temp\ipykernel_38408\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Goût médiocre, mousse ne tient pas, potentiellement contrefait.',
 'en_short_review': 'Poor taste, weak foam retention, potentially counterfeit.',
 'rating': 'I would rate the translation a 9 out of 10. The translation accurately conveys the original meaning and maintains the integrity of the message. The only slight improvement could be in the phrasing of "weak foam retention" to "poor foam stability" to better capture the original wording.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [65]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [66]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [67]:
llm = ChatOpenAI(temperature=0)

In [68]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [69]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [71]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [72]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [73]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\mktmi\AppData\Local\Temp\ipykernel_38408\3038952769.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [77]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")



> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.


In [78]:
response

{'input': 'Explain what principle component analysis is using linear algebra',
 'text': 'Principal Component Analysis (PCA) is a technique used in data analysis and machine learning to reduce the dimensionality of a dataset while preserving as much variance as possible. In linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset.\n\nThe covariance matrix is a square matrix that represents the relationships between the different features in the dataset. The eigenvectors of the covariance matrix represent the directions in which the data varies the most, while the eigenvalues represent the amount of variance in each of these directions.\n\nBy finding the eigenvectors and eigenvalues of the covariance matrix, PCA allows us to identify the principal components of the dataset - the directions in which the data varies the most. These principal components can then be used to transform the original dataset into a new, lower-dimensional

In [79]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. As the temperature of the black body increases, the intensity of the radiation increases and the peak wavelength of the radiation shifts to shorter wavelengths. This phenomenon is described by Planck's law of black body radiation."}

{'input': 'Explain what principle component analysis is using linear algebra',
 'text': 'Principal Component Analysis (PCA) is a technique used in data analysis and machine learning to reduce the dimensionality of a dataset while preserving as much variance as possible. In linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset.\n\nThe covariance matrix is a square matrix that represents the relationships between the different features in the dataset. The eigenvectors of the covariance matrix represent the directions in which the data varies the most, while the eigenvalues represent the amount of variance in each of these directions.\n\nBy finding the eigenvectors and eigenvalues of the covariance matrix, PCA allows us to identify the principal components of the dataset - the directions in which the data varies the most. These principal components can then be used to transform the original dataset into a new, lower-dimensional

In [82]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}

In [83]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**